In [47]:
import pandas as pd
import numpy as np
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import accuracy_score

In [2]:
def extract_time_features(df):
    df = df.copy()
    df["publishedAt"] = pd.to_datetime(df["publishedAt"])

    df["publish_hour"] = df["publishedAt"].dt.hour
    df["publish_day_of_week"] = df["publishedAt"].dt.dayofweek
    df["is_weekend"] = df["publish_day_of_week"].isin([5, 6]).astype(int)

    return df


In [3]:
def title_meta_features(series):
    return np.c_[
        series.str.len().fillna(0),
        series.str.split().apply(len).fillna(0),
        series.str.contains("!").astype(int),
        series.str.contains(r"\?").astype(int),
    ]


In [4]:
text_pipeline = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(
        max_features=8000,
        ngram_range=(1, 2),
        stop_words="english"
    ))
])

title_meta_pipeline = Pipeline(steps=[
    ("meta", FunctionTransformer(title_meta_features, validate=False))
])

In [5]:
df = pd.read_csv('../../../Dataset/Final_Dataset.csv')
df = extract_time_features(df)

C:\Users\mahmi\AppData\Local\Temp\ipykernel_17296\3784816730.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Dataset/Final_Dataset.csv')


In [6]:
df = df.sort_values("publishedAt").reset_index(drop=True)

In [7]:
split_idx = int(len(df) * 0.8)

train_df = df.iloc[:split_idx]
test_df  = df.iloc[split_idx:]

In [8]:
X_train = train_df.drop(columns=["is_trending", "publishedAt"])
y_train = train_df["is_trending"]

X_test = test_df.drop(columns=["is_trending", "publishedAt"])
y_test = test_df["is_trending"]

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("title_tfidf", text_pipeline, "title"),
        ("title_meta", title_meta_pipeline, "title"),
        ("time_features", "passthrough",
         ["publish_hour", "publish_day_of_week", "is_weekend"]),
        ("category", OneHotEncoder(handle_unknown="ignore"), ["categoryId"]),
        ("comments_disabled", "passthrough", ["comments_disabled"]),
    ],
    remainder="drop"
)


In [10]:
lgbm_model = LGBMClassifier(
    objective="binary",
    boosting_type="gbdt",
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    is_unbalance=True,   # veri dengesizliği için
    random_state=42,
    n_jobs=-1
)


In [11]:
model_pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", lgbm_model)
])

In [12]:
model_pipeline.fit(X_train, y_train)
joblib.dump(model_pipeline, "model_pipeline.pkl")

[LightGBM] [Info] Number of positive: 167238, number of negative: 144796
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.248093 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 247929
[LightGBM] [Info] Number of data points in the train set: 312034, number of used features: 8024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.535961 -> initscore=0.144092
[LightGBM] [Info] Start training from score 0.144092


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('title_tfidf', ...), ('title_meta', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the diff

In [68]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score
)

y_pred = model_pipeline.predict(X_test)
y_proba = model_pipeline.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"ROC-AUC  : {roc_auc:.4f}")
print(f"PR-AUC   : {pr_auc:.4f}")

C:\Users\mahmi\Desktop\VMP\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\mahmi\Desktop\VMP\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy : 0.8548
Precision: 0.1057
Recall   : 0.2077
F1-score : 0.1401
ROC-AUC  : 0.5875
PR-AUC   : 0.0794


In [67]:
# Bilgi girerek test etme
single_video = {
    "title": "faw",
    "categoryId": 1,
    "comments_disabled": 0,

    "publish_hour": 0,
    "publish_day_of_week": 6,
    "is_weekend": 1
}

single_video_df = pd.DataFrame([single_video])
trend_probability = model_pipeline.predict_proba(single_video_df)[0, 1]

print(f"Trending probability: %{trend_probability * 100:.2f}")

Trending probability: %72.27


C:\Users\mahmi\Desktop\VMP\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
